### Import relevant Libraries

In [34]:
from typing import Union, Dict
from enum import Enum
import os


from dotenv import load_dotenv
import openai
from openai import AzureOpenAI
from openai import OpenAI
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, LongType

openai.__version__

StatementMeta(, 9b036f43-6cae-488f-af66-679b1fec9828, 38, Finished, Available, Finished)

'1.51.0'

### Load credentials from `.env` file

In [21]:
load_dotenv("/lakehouse/default/Files/config_files/delta_ai.env")

StatementMeta(, 9b036f43-6cae-488f-af66-679b1fec9828, 25, Finished, Available, Finished)

True

### Initialize credentials

In [35]:
ENDPOINT = os.getenv("BASE_URL")
API_KEY = os.getenv("OPENAI_API_KEY")
API_VERSION = os.getenv("API_VERSION")
MODEL_NAME ="gpt-4o"#os.getenv("MODEL_NAME")


openai.api_key = API_KEY
openai.api_base = ENDPOINT
openai.api_type = "azure"
openai.api_version = API_VERSION

StatementMeta(, 9b036f43-6cae-488f-af66-679b1fec9828, 39, Finished, Available, Finished)

### Initialize the DataFrame

In [8]:
DELTA_TABLE_PATH = "user_data"
df = spark.read.table(DELTA_TABLE_PATH)

display(df)


StatementMeta(, 9b036f43-6cae-488f-af66-679b1fec9828, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 10fc364e-881a-44c5-a05b-be859940ec2b)

### Create the prompts used for the LLM

In [9]:
class PromptTemplate(Enum):

    tableSChema = """
    - 'Name' StringType(): Name of the person registered on the database 
    - 'Gender' StringType(): The gender of the person registered on the database
    - 'Id' StringType(): The unique ID of the person registered on the database
    - 'Age' LongType(): The age of the person registered on the database
    - 'Date_of_birth' DateType(): The date of birth of the person registered on the database
    - 'Address' StringType(): The address of the person registered on the database
    - 'Timezone_location' StringType(): The Timezone location which is the Name of a country or a city
    - 'Timezone_offset' StringType(): The number of hours ahead or behind GMT but it is a string
    - 'Hours_from_gmt' DoubleType(): The number of hours ahead or behind GMT and it is a number
    - 'Longitude' DoubleType(): The Longitude of the person's current location
    - 'Latitude' DoubleType(): The latitude of the person's current location
    - 'Cell' StringType(): The cell phone number of the person registered on the database
    - 'Phone' StringType(): The telephone number of the person registered on the database
    - 'Email' StringType(): The email of the person registered
    - 'Registered_date' DateType(): The date the person got registered on the application
    - 'Picture_url' StringType(): A URL Link to the person's picture
    - 'Insertion_time' StringType(): The datetime that the person registered on the application
    - 'Ingestion_Date' TimestampType(): The datetime when data was ingested into the current database
    """
    

    selectTable = """
    
    You are provided with a Delta Table called {table_name} that contains the following schema:

    schema = {schema}

    Here is a sample row from the table:
    ("Ms Maiara Pires", "female", 48, '1975-09-26', "6712 Avenida Vinícius de Morais, Águas Lindas de Goiás, Distrito Federal - Brazil 63002", 
    "Abu Dhabi, Muscat, Baku, Tbilisi", "+4:00", 4.0, 86.4685, -81.2552, "(06) 4562-0948", "(97) 1291-9452", "maiara.pires@example.com", '1975-09-26', 
    "https://randomuser.me/api/portraits/women/4.jpg", '2024-09-22 20:02:12.878474', '2024-09-22 20:03:12.878474')

    Please answer the following question using SQL syntax. Your response should fit into this Spark SQL command:

    ```python
    df = spark.sql(<query>)
    ```

    The SQL query should answer the question below based on the provided schema and data:
    
    Question: {prompt}

    Ensure that:
    - Ensure that the query is valid for use with Spark SQL.
    - If the question is related to the table, you return the appropriate SQL query without any newline or tab characters. Do not use backticks
    - If the question has nothing to do with the table or the schema, return an empty string (`""`).
    

    Lastly give me only the query as your response. No extra formatting, just the raw query alone. Nothing more. 
    If the prompt is inadequete to generate a query, return an empty string
    """


    summarizeTable = """
    I have the results of a query executed on a Delta table named {table_name}. Due to potential size constraints, not all results are provided. The query results are summarized in the following Python dictionary:

    query_result = {prompt_result}

    This result corresponds to the following question: {prompt}

    The schema of the original table is as follows:
    {schema}

    Please provide a concise summary of the query_result, taking into account the question asked and the provided schema. Your summary should highlight key insights, trends, or notable findings relevant to the query.

    Aim for a response length between 20 and 200 words. Ensure clarity and coherence in your summary, and note that you may need to focus on the most relevant aspects given the limited data.

    A shorter response is always better
    """

    badPromptError = "No DataFrame generated. This might be due to a bad prompt. Check gpt response variable"



StatementMeta(, 9b036f43-6cae-488f-af66-679b1fec9828, 13, Finished, Available, Finished)

### Create the functions used to interact with the LLM

In [36]:

def get_openai_client(*, endpoint: str, api_key: str, api_version: str)-> AzureOpenAI:
    """
    Initializes and returns an Azure OpenAI client.

    This function creates an instance of the AzureOpenAI client using 
    the provided endpoint, API key, and API version.

    Parameters:
    endpoint (str): The Azure endpoint for the OpenAI service.
    api_key (str): The API key for authenticating with the Azure OpenAI service.
    api_version (str): The version of the API to use.

    Returns:
    AzureOpenAI: An instance of the AzureOpenAI client configured with the provided parameters.
    """
    
    client = AzureOpenAI(
    azure_endpoint= endpoint,
    api_key= api_key,
    api_version= api_version,
    )    

    return client



def instruction_type_select(*, instruction_type)-> Enum:
    """
    Selects the appropriate instruction type based on the given input.

    This function maps instruction types to their corresponding prompt templates.

    Parameters:
    instruction_type (str): The type of instruction to select (e.g., "select" or "summarize").

    Returns:
    Enum: The corresponding prompt template for the specified instruction type.

    Raises:
    KeyError: If the instruction_type is not found in the mapping.
    """

    instruction_types = {
        "select": PromptTemplate.selectTable,
        "summarize": PromptTemplate.summarizeTable
        }

    return instruction_types[instruction_type]



def generate_instruction(*, prompt: str, table_name: str, prompt_result: Union[Dict, None] = None)-> str:
    """
    Generates an instruction string based on the provided prompt and table information.

    This function creates an instruction string for querying or summarizing a table 
    using the specified prompt, table name, and optional prompt results.

    Parameters:
    prompt (str): The prompt string to be used for instruction generation.
    table_name (str): The name of the table related to the instruction.
    prompt_result (Union[Dict, None], optional): The result of a previous prompt, if any.

    Returns:
    str: The generated instruction string. If the prompt result is empty or invalid, 
         returns an error message.
    """

    table_schema: str = PromptTemplate.tableSChema.value
    
    if prompt_result == {}:
        return PromptTemplate.badPromptError.value

    if not prompt_result:
        instruction: Enum = instruction_type_select(instruction_type = "select")
        instruction: str = instruction.value.format(prompt = prompt, table_name = table_name, schema = table_schema)
    else:
        instruction: Enum = instruction_type_select(instruction_type = "summarize")
        instruction: str = instruction.value.format(prompt_result = prompt_result, 
                                                    prompt = prompt, 
                                                    table_name = table_name, 
                                                    schema = table_schema
                                                    )

    return instruction


    
def get_gpt_response(client: AzureOpenAI, prompt: str, table_name: str, prompt_result: Union[Dict, None] = None, model_name: str = MODEL_NAME, temperature: float = 0.0)-> str:
    """
    Retrieves a response from the GPT model using the provided instruction.

    This function generates an instruction based on the prompt and other parameters, 
    then sends it to the Azure OpenAI client to obtain a completion.

    Parameters:
    client (AzureOpenAI): The Azure OpenAI client to use for making requests.
    prompt (str): The prompt string for generating the instruction.
    table_name (str): The name of the table related to the instruction.
    prompt_result (Union[Dict, None], optional): The result of a previous prompt, if any.
    model_name (str, optional): The name of the model to use for the completion. Defaults to MODEL_NAME.
    temperature (float, optional): The temperature for the model's response variability. Defaults to 0.0.

    Returns:
    str: The content of the model's response, or an error message if the instruction is invalid.
    """

    instruction = generate_instruction(prompt = prompt, table_name = table_name, prompt_result = prompt_result)

    if instruction == PromptTemplate.badPromptError.value:
        return instruction

    MESSAGES = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": instruction},
    ]

    completion = client.chat.completions.create(
        model=model_name,
        messages=MESSAGES,
        temperature=0
    )
    return completion.choices[0].message.content



def parse_gpt_response(response: str)-> str:
    """
    Parses the GPT response to extract the relevant SQL query.

    This function attempts to extract the SQL query from the GPT response string. 
    If the response is not in the expected format, an error message is printed.

    Parameters:
    response (str): The response string from the GPT model.

    Returns:
    str: The extracted SQL query, or None if the query cannot be parsed.
    """
    try:
        response = response.strip("\n").replace("sql", "").replace("\n", " ").replace("\t", "").strip("`")
    except IndexError:
        print("Response has no SQL query")
        return
    else:
        return response



def generate_gpt_dataframe(parsed_response: str)-> Union[str, DataFrame]:
    """
    Generates a DataFrame from a parsed GPT response query.

    This function takes a parsed response, which is expected to be a SQL query string, 
    and executes it on a Spark session to return the resulting DataFrame. If the response 
    is invalid (i.e., the parsed response is empty), it returns an error message instead 
    of the DataFrame.

    Args:
        parsed_response (str): The SQL query string generated from the GPT response.

    Returns:
        Union[str, DataFrame]: The resulting Spark DataFrame from the SQL query, or an 
        error message string if the query is invalid.
    """

    if parsed_response:
        gpt_df = spark.sql(parsed_response)
    else:
        gpt_df = PromptTemplate.badPromptError.value
    return gpt_df



def convert_gpt_dataframe_to_dict(gpt_df: DataFrame)-> Dict:
    """
    Converts a Spark DataFrame to a dictionary representation.

    This function converts the given Spark DataFrame into a Pandas DataFrame and 
    extracts its first 5 rows into a dictionary, using a 'list' orientation. If the 
    DataFrame represents an error (e.g., due to an invalid query), it returns an 
    empty dictionary.

    Args:
        gpt_df (DataFrame): The Spark DataFrame generated from the parsed GPT query.

    Returns:
        Dict: A dictionary containing the first 5 rows of the DataFrame, where each 
        column is a key and the values are lists of data for that column. Returns 
        an empty dictionary if the DataFrame is invalid.
    """
    
    if gpt_df == PromptTemplate.badPromptError.value:
        return {}
    pandas_df = gpt_df.toPandas()
    pandas_dict = pandas_df.head(5).to_dict(orient="list")

    return pandas_dict

StatementMeta(, 9b036f43-6cae-488f-af66-679b1fec9828, 40, Finished, Available, Finished)

### Test running the LLM query responses

In [43]:
#prompt = "Which timezone is the farthest from GMT in the database and how far is it in hours?"#"I don't like Donald Trump"#
prompt = "How many unique locations are there and how far are they from GMT?"#"I don't like Donald Trump"#

openai_client = get_openai_client(endpoint = ENDPOINT, api_key = API_KEY, api_version = API_VERSION)

gpt_response = get_gpt_response(openai_client, prompt = prompt, table_name = "user_data")

parsed_response = parse_gpt_response(gpt_response)

gpt_df = generate_gpt_dataframe(parsed_response)

gpt_dict = convert_gpt_dataframe_to_dict(gpt_df)

gpt_summary = get_gpt_response(openai_client, prompt = prompt, table_name = "user_data", prompt_result= gpt_dict)

StatementMeta(, 9b036f43-6cae-488f-af66-679b1fec9828, 47, Finished, Available, Finished)

In [44]:
gpt_summary

StatementMeta(, 9b036f43-6cae-488f-af66-679b1fec9828, 48, Finished, Available, Finished)

'The query results indicate that there are five unique timezone locations in the user_data table, each with a distinct offset from GMT. These locations include "Pacific Time (US & Canada)" at -8.0 hours, "Abu Dhabi, Muscat, Baku, Tbilisi" at +4.0 hours, "Tehran" at +3.5 hours, "Alaska" at -9.0 hours, and "Mountain Time (US & Canada)" at -7.0 hours. This diversity in timezone locations suggests a geographically varied user base, spanning multiple continents and time zones. The data highlights the global reach of the user base, with time differences ranging from -9.0 to +4.0 hours relative to GMT.'

In [45]:
display(gpt_df)

StatementMeta(, 9b036f43-6cae-488f-af66-679b1fec9828, 49, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fe5ff0ab-3867-46d1-80f7-8a998b859d0d)

In [10]:

# #prompt = "Which timezone is the farthest from GMT in the database and how far is it in hours?"#"I don't like Donald Trump"#
# prompt = "How many unique locations are there and how far are they from GMT?"#"I don't like Donald Trump"#
# API_KEY = "sk-YqHM00e4OYMM2VOzVHpzbyx-BOU-b4CXaAWwmcVuMsT3BlbkFJMT1Vd2wnyTkaOIZQTlLgFEC7hkvr33Rerg75l-sAsA"
# openai_client = OpenAI(api_key=API_KEY)#get_openai_client(endpoint = ENDPOINT, api_key = API_KEY, api_version = API_VERSION)

# gpt_response = get_gpt_response(openai_client, prompt = prompt, table_name = "user_data")

# parsed_response = parse_gpt_response(gpt_response)

# gpt_df = generate_gpt_dataframe(parsed_response)

# gpt_dict = convert_gpt_dataframe_to_dict(gpt_df)

# gpt_summary = get_gpt_response(openai_client, prompt = prompt, table_name = "user_data", prompt_result= gpt_dict)

StatementMeta(, cc250cdc-0c94-444e-91a6-21f1f749285c, 14, Finished, Available, Finished)